In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from xgboost.spark import SparkXGBClassifier
# from xgboost import XGBoostClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col
from sklearn.metrics import f1_score, recall_score, classification_report

In [2]:
def init_spark():
    spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()
    return spark
spark_object = init_spark()

23/03/29 20:10:15 WARN Utils: Your hostname, karyubuntu resolves to a loopback address: 127.0.1.1; using 172.31.170.192 instead (on interface wlp0s20f3)
23/03/29 20:10:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/29 20:10:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/29 20:10:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/29 20:10:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
train_spark = spark_object.read.option("delimiter", ",").csv("./data/trainbalanced.csv", header='true', inferSchema='true')
test_spark = spark_object.read.option("delimiter", ",").csv("./data/testbalanced.csv", header='true', inferSchema='true')

In [4]:
for i in train_spark.columns:
    oldname = i
    if "." in oldname:
        newname = oldname.replace(".","")
        train_spark = train_spark.withColumnRenamed(oldname, newname)
        test_spark = test_spark.withColumnRenamed(oldname, newname)

In [5]:
# train_data, test_data = train_spark.randomSplit([0.7, 0.3], seed=123)

In [5]:
input_list =[]
for i in train_spark.columns:
    if i != 'isFraud':
        input_list.append(i)

In [6]:
assembler = VectorAssembler(inputCols=input_list,outputCol='features')

In [7]:
train_features = assembler.transform(train_spark)
test_features = assembler.transform(test_spark)

In [8]:
xgb_models=[]
num_rounds=[10, 50, 100]
max_depths = [3, 10, 20]
for nr in num_rounds:
    for md in max_depths:
        print("-"*25)
        print("num_rounds: " + str(nr))
        print("max_depth: " + str(md))
        dt = SparkXGBClassifier(
            features_col="features",
            label_col="isFraud",
            num_round=nr,
            max_depth=md,
            eta=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            seed=42
        )
        model = dt.fit(train_features)
        predictions = model.transform(train_features)
        y_train_pred=predictions.select("prediction").collect()
        y_train_orig=predictions.select("isFraud").collect()
        print('Training Report')
        print(classification_report(y_train_orig, y_train_pred))
        
        predictions = model.transform(test_features)
        y_test_pred=predictions.select("prediction").collect()
        y_test_orig=predictions.select("isFraud").collect()
        print('Test Report')
        print(classification_report(y_test_orig, y_test_pred))
        xgb_models.append(model)


-------------------------
num_rounds: 10
max_depth: 3
23/03/29 20:10:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[20:10:42] task 0 got new rank 0                                    (0 + 1) / 1]
[20:10:45] WARNING: ../src/learner.cc:767: 
Parameters: { "num_round" } are not used.

/home/kary/anaconda3/envs/bigdata/lib/python3.10/site-packages/xgboost/sklearn.py:782: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  warnings.warn("Loading a native XGBoost model with Scikit-Learn interface.")


Training Report
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.97     93062
         1.0       0.98      0.95      0.97     93062

    accuracy                           0.97    186124
   macro avg       0.97      0.97      0.97    186124
weighted avg       0.97      0.97      0.97    186124



Test Report
              precision    recall  f1-score   support

         0.0       0.97      0.98      0.97     39853
         1.0       0.75      0.59      0.66      3417

    accuracy                           0.95     43270
   macro avg       0.86      0.79      0.82     43270
weighted avg       0.95      0.95      0.95     43270

-------------------------
num_rounds: 10
max_depth: 10


[20:15:15] task 0 got new rank 0                                    (0 + 1) / 1]
[20:15:18] WARNING: ../src/learner.cc:767: 
Parameters: { "num_round" } are not used.

/home/kary/anaconda3/envs/bigdata/lib/python3.10/site-packages/xgboost/sklearn.py:782: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  warnings.warn("Loading a native XGBoost model with Scikit-Learn interface.")


Training Report
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     93062
         1.0       1.00      0.98      0.99     93062

    accuracy                           0.99    186124
   macro avg       0.99      0.99      0.99    186124
weighted avg       0.99      0.99      0.99    186124



Test Report
              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98     39853
         1.0       0.89      0.70      0.78      3417

    accuracy                           0.97     43270
   macro avg       0.93      0.85      0.88     43270
weighted avg       0.97      0.97      0.97     43270

-------------------------
num_rounds: 10
max_depth: 20


23/03/29 20:21:31 ERROR PythonRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/kary/anaconda3/envs/bigdata/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 666, in main
    eval_type = read_int(infile)
  File "/home/kary/anaconda3/envs/bigdata/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:765)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:747)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator.fore

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Could not recover from a failed barrier ResultStage. Most recent failure reason: Stage failed because barrier task ResultTask(29, 0) finished unsuccessfully.
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/kary/anaconda3/envs/bigdata/lib/python3.10/site-packages/xgboost/spark/core.py", line 817, in _train_booster
    _rabit_args = _get_rabit_args(context, num_workers)
  File "/home/kary/anaconda3/envs/bigdata/lib/python3.10/site-packages/xgboost/spark/utils.py", line 77, in _get_rabit_args
    env = _start_tracker(context, n_workers)
  File "/home/kary/anaconda3/envs/bigdata/lib/python3.10/site-packages/xgboost/spark/utils.py", line 63, in _start_tracker
    rabit_context = RabitTracker(host_ip=host, n_workers=n_workers)
  File "/home/kary/anaconda3/envs/bigdata/lib/pyth

In [ ]:
# for model in xgb_models:
#     try:
#         print("-"*25)
#         print("Depth: " + str(md))
#         print("numTrees: " + str(nt))
#         model = model.fit(train_features)
#         predictions = model.transform(test_features)
#         y_pred=predictions.select("prediction").collect()
#         y_orig=predictions.select("isFraud").collect()
#         classification_report(y_orig, y_pred)
#     except Exception as e:
#         print(e)

In [ ]:
# xgb = SparkXGBClassifier(
#     features_col="features",
#     label_col="isFraud",
#     num_round=10,
#     max_depth=3,
#     eta=0.1,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     seed=42
# )

In [ ]:
# xgb = SparkXGBClassifier(label_col="cnt",max_depth=5, missing=0.0,
#     validation_indicator_col='isVal', weight_col='weight',
#     early_stopping_rounds=1, eval_metric='logloss')

# new_xgb = SparkXGBClassifier(max_depth=5, missing=0.0,
# validation_indicator_col='isVal', weight_col='weight',
# early_stopping_rounds=1, eval_metric='logloss')

In [ ]:
# try:
#     model = xgb.fit(train_features)
#     predictions = model.transform(test_features)
# except Exception as e:
#     print(e)

In [ ]:
# predictions = model.transform(test_features)

In [ ]:
# y_pred=predictions.select("prediction").collect()
# y_orig=predictions.select("isFraud").collect()

In [ ]:
# print(f1_score(y_orig, y_pred))
# print(recall_score(y_orig, y_pred))

In [ ]:
# evaluator = BinaryClassificationEvaluator(labelCol="isFraud")
# accuracy = evaluator.evaluate(predictions)

# print(f"Accuracy: {accuracy:.4f}")